In [190]:
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px 
import glob
import numpy as np

In [191]:
NODE_COUNT=4
USE_META_LEARNING=False

In [192]:
if USE_META_LEARNING:
    results = [r for r in glob.glob(f"./../results/*AVERAGE_AGGREGATION_{NODE_COUNT}*/") if not "CLIPPED" in r]
    plot_file_term="MetaL_method"
else:
    results = glob.glob(f"./../results/NO_META_LEARNING*{NODE_COUNT}*/")
    plot_file_term="FL_method"
len(results)

8

In [193]:
results

['./../results/NO_META_LEARNING_MEDIAN_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_RECENCY_4_NODES/',
 './../results/NO_META_LEARNING_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_PERFORMANCE_SCORES_4_NODES/',
 './../results/NO_META_LEARNING_NO_FEDERATED_LEARNING_4_NODES/',
 './../results/NO_META_LEARNING_FIXED_CLIPPED_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_PUNISHING_UPDATES_4_NODES/']

In [194]:
iter_dfs = [pickle.load(open(file +"/iter_df.pkl","rb")) for file in results]
training_dfs = [pickle.load(open(file +"/training_df.pkl","rb")) for file in results]

if not USE_META_LEARNING:
    aggregation_methods=["_".join(file.split("_")[3:-2]) for file in results]
else:
    aggregation_methods=["_".join(file.split("/")[-2].split("_")[:-4]) for file in results]

In [195]:
aggregation_methods

['MEDIAN_AGGREGATION',
 'DYNAMIC_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_RECENCY',
 'AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PERFORMANCE_SCORES',
 'NO_FEDERATED_LEARNING',
 'FIXED_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PUNISHING_UPDATES']

In [196]:
iter_dfs[0]

,curr_score_0,iter_counter_0,DG_rotation_0,timestamp_0,curr_score_1,iter_counter_1,DG_rotation_1,timestamp_1,curr_score_2,iter_counter_2,DG_rotation_2,timestamp_2,curr_score_3,iter_counter_3,DG_rotation_3,timestamp_3
0,NaN,1,1,2024-02-27 13:29:02.596435,NaN,1,1,2024-02-27 13:29:02.045701,NaN,1,1,2024-02-27 13:29:02.518169,NaN,1,0,2024-02-27 13:29:01.795107
1,0.740667,2,2,2024-02-27 13:29:21.218587,0.784333,2,1,2024-02-27 13:29:22.183886,0.784333,2,1,2024-02-27 13:29:19.823487,0.781667,2,0,2024-02-27 13:29:13.157115
2,0.737333,3,2,2024-02-27 13:29:21.944798,0.782333,3,1,2024-02-27 13:29:22.914995,0.784333,3,1,2024-02-27 13:29:20.602388,0.780000,3,0,2024-02-27 13:29:13.957649
3,0.751667,4,3,2024-02-27 13:29:22.684449,0.788000,4,1,2024-02-27 13:29:23.643037,0.798333,4,2,2024-02-27 13:29:21.333011,0.786333,4,0,2024-02-27 13:29:14.746694
4,0.752667,5,3,2024-02-27 13:29:23.444855,0.786000,5,1,2024-02-27 13:29:24.382645,0.802667,5,2,2024-02-27 13:29:22.056329,0.787333,5,0,2024-02-27 13:29:15.538558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.797333,996,627,2024-02-27 13:52:50.803471,0.759667,996,218,2024-02-27 13:46:21.857217,0.799333,996,280,2024-02-27 13:51:12.286472,0.782000,996,197,2024-02-27 13:45:51.956932
996,0.796000,997,627,2024-02-27 13:52:51.553904,0.766333,997,218,2024-02-27 13:46:22.626554,0.798000,997,280,2024-02-27 13:51:13.046783,0.779333,997,197,2024-02-27 13:45:52.748490
997,0.801333,998,627,2024-02-27 13:52:52.282530,0.763000,998,218,2024-02-27 13:46:23.377351,0.793333,998,281,2024-02-27 13:51:13.808405,0.779667,998,198,2024-02-27 13:45:53.613401
998,0.800333,999,627,2024-02-27 13:52:53.038111,0.761667,999,218,2024-02-27 13:46:24.152903,0.784000,999,282,2024-02-27 13:51:14.577367,0.781333,999,198,2024-02-27 13:45:54.397592


In [197]:
scores=[]
for aggregation_method,iter_df in zip(aggregation_methods,iter_dfs):
    rel_columns= [c for c in iter_df.columns if "curr_score_" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = iter_df[rel_columns]
    scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [198]:
score_df=pd.concat(scores,axis=1)
score_df

,MEDIAN_AGGREGATION_curr_score_0,MEDIAN_AGGREGATION_curr_score_1,MEDIAN_AGGREGATION_curr_score_2,MEDIAN_AGGREGATION_curr_score_3,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_0,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_1,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_2,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_3,WEIGHTED_AVERAGE_RECENCY_curr_score_0,WEIGHTED_AVERAGE_RECENCY_curr_score_1,...,NO_FEDERATED_LEARNING_curr_score_2,NO_FEDERATED_LEARNING_curr_score_3,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_0,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_1,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_2,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_3,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_0,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.740667,0.784333,0.784333,0.781667,0.740667,0.784333,0.781667,0.784333,0.784333,0.781667,...,0.781667,0.784333,0.781667,0.784333,0.784333,0.740667,0.740667,0.781667,0.784333,0.784333
2,0.737333,0.782333,0.784333,0.780000,0.737333,0.784333,0.780000,0.782333,0.782333,0.780000,...,0.780000,0.782333,0.780000,0.784333,0.782333,0.737333,0.737333,0.780000,0.784333,0.782333
3,0.751667,0.788000,0.798333,0.786333,0.751667,0.798333,0.786333,0.788000,0.788000,0.786333,...,0.786333,0.788000,0.786333,0.798333,0.788000,0.751667,0.751667,0.786333,0.798333,0.788000
4,0.752667,0.786000,0.802667,0.787333,0.752667,0.802667,0.787333,0.786000,0.786000,0.787333,...,0.787333,0.786000,0.787333,0.802667,0.786000,0.752667,0.752667,0.787333,0.802667,0.786000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.797333,0.759667,0.799333,0.782000,0.797333,0.749333,0.677000,0.666667,0.794667,0.774667,...,0.818333,0.841667,0.797667,0.760667,0.549333,0.794667,0.500000,0.651333,0.500000,0.660000
996,0.796000,0.766333,0.798000,0.779333,0.793667,0.748000,0.674667,0.665667,0.798000,0.777000,...,0.819667,0.834000,0.800667,0.760333,0.500000,0.793333,0.500000,0.647667,0.500000,0.660000
997,0.801333,0.763000,0.793333,0.779667,0.795333,0.730000,0.672667,0.665667,0.796000,0.762000,...,0.813667,0.840667,0.792333,0.735333,0.712000,0.795667,0.500000,0.656000,0.500000,0.659000
998,0.800333,0.761667,0.784000,0.781333,0.797333,0.713667,0.672667,0.666667,0.801000,0.765000,...,0.811667,0.834000,0.795667,0.712667,0.716333,0.793667,0.500000,0.651333,0.500000,0.661667


In [199]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    rel_col=[c for c in score_df.columns if aggregation_method == c[:-13]]
    result_dict=dict()
    by_node_result_average_dict=dict()
    for col in rel_col:
        by_node_result_average_dict[col.split("_")[-1]]=score_df[col].mean()
    result_dict["by_node_results"]=score_df[rel_col].mean(axis=0)
    result_dict["by_node_results_std"]=score_df[rel_col].std(axis=0)
    result_dict["total_average"]=score_df[rel_col].mean().mean()
    result_dict["total_std"]=score_df[rel_col].mean().std()
    result_dict["average_in_time"]= score_df[rel_col].mean(axis=1)
    average_score_dict[aggregation_method]=result_dict

In [200]:
aggregation_names=[]
aggregation_averages=[]
aggregation_stds=[]
by_node_averages=[]
by_node_stds=[]
average_in_time=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    aggregation_averages.append(val["total_average"])
    aggregation_stds.append(val["total_std"])
    by_node_averages.append(val["by_node_results"].values)
    by_node_stds.append(val["by_node_results_std"])
    average_in_time.append(val["average_in_time"])
sorted_tuples=sorted(zip(aggregation_names,aggregation_averages,by_node_averages,average_in_time,aggregation_stds, by_node_stds),key=lambda x: x[1])
aggregation_names = [val[0] for val in sorted_tuples]
aggregation_averages = [val[1] for val in sorted_tuples]
by_node_averages = [val[2] for val in sorted_tuples]
average_in_time = [val[3] for val in sorted_tuples]
aggregation_stds = [val[4] for val in sorted_tuples]
by_node_stds = [val[5] for val in sorted_tuples]

In [201]:
n_nodes=len(by_node_averages[0])
n_nodes

4

In [202]:
by_node_averages

[array([0.72835335, 0.71352553, 0.73285352, 0.67662696]),
 array([0.7076373 , 0.72054121, 0.72524157, 0.71779379]),
 array([0.68130998, 0.74039072, 0.70618852, 0.75170571]),
 array([0.73338105, 0.71225893, 0.72534835, 0.71106006]),
 array([0.72108075, 0.7158622 , 0.7475352 , 0.74295395]),
 array([0.77361562, 0.77614147, 0.72694595, 0.73555622, 0.71941441,
        0.75466066, 0.72180581, 0.708671  ]),
 array([0.76564998, 0.77342109, 0.71827294, 0.75063931]),
 array([0.78987955, 0.78439673, 0.79038839, 0.78621955])]

In [203]:
processed_aggregation_names=[" ".join(name.split("_")[:-1]) for name in aggregation_names]
processed_aggregation_names

['FIXED CLIPPED AVERAGE',
 'MEDIAN',
 'WEIGHTED AVERAGE PUNISHING',
 'AVERAGE',
 'DYNAMIC CLIPPED AVERAGE',
 'WEIGHTED AVERAGE PERFORMANCE',
 'WEIGHTED AVERAGE',
 'NO FEDERATED']

In [204]:
aggregation_names

['FIXED_CLIPPED_AVERAGE_AGGREGATION',
 'MEDIAN_AGGREGATION',
 'WEIGHTED_AVERAGE_PUNISHING_UPDATES',
 'AVERAGE_AGGREGATION',
 'DYNAMIC_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PERFORMANCE_SCORES',
 'WEIGHTED_AVERAGE_RECENCY',
 'NO_FEDERATED_LEARNING']

In [205]:
name_mapping={"FIXED_CLIPPED_AVERAGE_AGGREGATION": "Fixed clip",
 "MEDIAN_AGGREGATION": "Median",
 "DYNAMIC_CLIPPED_AVERAGE_AGGREGATION": "Dynamic clip",
 "WEIGHTED_AVERAGE_PUNISHING_UPDATES": "Punish updates",
 "AVERAGE_AGGREGATION": "Simple average",
 "WEIGHTED_AVERAGE_RECENCY": "Receny weighted",
 "WEIGHTED_AVERAGE_PERFORMANCE_SCORES": "Performance weighted",
 "NO_FEDERATED_LEARNING": "No FL",
 "NO_META_LEARNING": "No MetaL",
 "DIRECT_SCORE_WEIGHTED_MEANS":"Direct weighted",
 "DIRECT_GENETIC_ALGORITHM_MATING":"Direct sampled",
 "GRADUAL_SCORE_WEIGHTED_MEANS":"Gradual weighted",
 "GRADUAL_MEANS":"Gradual regular",
 "GRADUAL_GENETIC_ALGORITHM_MATING":"Gradual sampled",
 "DIRECT_MEANS":"Direct regular",}
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Fixed clip',
 'Median',
 'Punish updates',
 'Simple average',
 'Dynamic clip',
 'Performance weighted',
 'Receny weighted',
 'No FL']

In [206]:
fig=go.Figure()
for node in range(n_nodes):
    fig.add_trace(go.Bar(x=processed_aggregation_names, y=[val[node] for val in by_node_averages],error_y=dict(type="data",array=[val[node] for val in by_node_stds]),name=f"Node {node}"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=aggregation_averages,error_y=dict(type="data",array=aggregation_stds),name="Average"))
fig.update_layout(template="none", title=f"Accuracy score by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Average_score_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

/tmp/ipykernel_873588/1946294216.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [207]:
fig=go.Figure()
for idx,name in enumerate(processed_aggregation_names):
    fig.add_trace(go.Scatter(y=average_in_time[idx][1:] ,name=name))
fig.update_layout(template="none", title=f"Accuracy score by {' '.join(plot_file_term.split('_'))} for {NODE_COUNT} nodes",
                  width=800,height=600,legend=dict( orientation="h",y=1.2),margin=dict(t=150,r=10))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Performance_in_time_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

In [208]:
training_dfs[0].head()

,batch_size_mean_0,batch_size_std_0,learning_rate_mean_0,learning_rate_std_0,nesterov_mean_0,nesterov_std_0,momentum_mean_0,momentum_std_0,n_runs_mean_0,n_runs_std_0,...,n_runs_3,n_epochs_3,weights_3,iter_count_3,pre_training_score_3,scores_3,best_score_3,best_hyper_params_3,best_model_weights_3,timestamp_3
0,64,20,-3,1,0.5,0.3,0.5,0.3,10,3,...,6.0,1.500000,"[[[-0.284298300743103, -0.3477519154548645, 0....",514.0,0.428333,"[0.43, 0.7783333333333333, 0.43166666666666664...",0.808333,"{'learning_rate': 0.006748051918868561, 'batch...","[[[-0.5424570441246033, -0.33346447348594666, ...",2024-02-27 13:36:07.666136
1,64,20,-3,1,0.5,0.3,0.5,0.3,10,3,...,10.0,1.000000,"[[[-0.284892201423645, -0.34779414534568787, 0...",596.0,0.461667,"[0.46166666666666667, 0.46, 0.4816666666666667...",0.773333,"{'learning_rate': 0.0012758149410259506, 'batc...","[[[-1.4805386066436768, -0.19684354960918427, ...",2024-02-27 13:37:40.665813
2,64,20,-3,1,0.5,0.3,0.5,0.3,10,3,...,9.0,0.444444,"[[[-0.17907551676034927, -0.32137130200862885,...",770.0,0.673333,"[0.6733333333333333, 0.7816666666666666, 0.511...",0.800000,"{'learning_rate': 0.009112882985346505, 'batch...","[[[0.3142632842063904, -0.3111160397529602, 1....",2024-02-27 13:41:36.113173
3,64,20,-3,1,0.5,0.3,0.5,0.3,10,3,...,10.0,0.800000,"[[[-0.47751791775226593, -0.33354710042476654,...",662.0,0.701667,"[0.4583333333333333, 0.5983333333333334, 0.816...",0.816667,"{'learning_rate': 0.002794828480690338, 'batch...","[[[-0.18695427477359772, -0.32392817735671997,...",2024-02-27 13:39:19.338050
4,64,20,-3,1,0.5,0.3,0.5,0.3,10,3,...,9.0,0.666667,"[[[-0.09235608577728271, -0.32445429265499115,...",805.0,0.681667,"[0.6816666666666666, 0.49666666666666665, 0.49...",0.806667,"{'learning_rate': 0.001132248821997721, 'batch...","[[[0.6123579144477844, -0.3018307089805603, 1....",2024-02-27 13:42:20.638013


In [209]:
trainig_scores=[]
for aggregation_method,training_df in zip(aggregation_methods,training_dfs):
    rel_columns= [c for c in training_df.columns if "score" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = training_df[rel_columns]
    trainig_scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [210]:
training_scores_df=pd.concat(trainig_scores,axis=1)
training_scores_df

,MEDIAN_AGGREGATION_pre_training_score_0,MEDIAN_AGGREGATION_scores_0,MEDIAN_AGGREGATION_best_score_0,MEDIAN_AGGREGATION_pre_training_score_1,MEDIAN_AGGREGATION_scores_1,MEDIAN_AGGREGATION_best_score_1,MEDIAN_AGGREGATION_pre_training_score_2,MEDIAN_AGGREGATION_scores_2,MEDIAN_AGGREGATION_best_score_2,MEDIAN_AGGREGATION_pre_training_score_3,...,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_0,WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_scores_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_scores_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_3,WEIGHTED_AVERAGE_PUNISHING_UPDATES_scores_3,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_3
0,0.403333,"[0.40166666666666667, 0.6616666666666666, 0.38...",0.760000,0.431667,"[0.815, 0.44166666666666665, 0.435, 0.80166666...",0.815000,0.486667,"[0.7783333333333333, 0.7866666666666666, 0.753...",0.788333,0.428333,...,0.510000,0.505000,"[0.7783333333333333, 0.5066666666666667, 0.495...",0.778333,0.475000,"[0.475, 0.475, 0.475, 0.475, 0.475, 0.475]",0.475000,0.781667,"[0.7966666666666666, 0.78, 0.7883333333333333,...",0.796667
1,0.661667,"[0.6733333333333333, 0.7916666666666666, 0.791...",0.791667,0.490000,"[0.49, 0.82, 0.8333333333333334, 0.48833333333...",0.833333,0.528333,"[0.5066666666666667, 0.73, 0.52, 0.61666666666...",0.785000,0.461667,...,0.518333,0.596667,"[0.615, 0.79, 0.69, 0.6233333333333333]",0.790000,0.508333,"[0.5083333333333333, 0.5083333333333333, 0.508...",0.508333,0.501667,"[0.49, 0.6083333333333333, 0.6083333333333333,...",0.766667
2,0.505000,"[0.8166666666666667, 0.81, 0.8166666666666667,...",0.816667,0.490000,"[0.69, 0.69, 0.79, 0.505, 0.4166666666666667, ...",0.798333,0.585000,"[0.5116666666666667, 0.5233333333333333, 0.506...",0.523333,0.673333,...,0.516667,0.663333,"[0.7833333333333333, 0.67, 0.6783333333333333,...",0.810000,0.503333,"[0.5033333333333333, 0.5033333333333333, 0.503...",0.503333,0.760000,"[0.755, 0.7466666666666667, 0.7466666666666667]",0.755000
3,0.398333,"[0.7566666666666667, 0.7716666666666666, 0.396...",0.771667,0.436667,"[0.4766666666666667, 0.44166666666666665, 0.58...",0.815000,0.475000,"[0.79, 0.475, 0.475, 0.43833333333333335, 0.75...",0.790000,0.701667,...,0.476667,0.420000,"[0.41, 0.4716666666666667, 0.41, 0.41, 0.77833...",0.778333,0.501667,"[0.5016666666666667, 0.5016666666666667, 0.501...",0.501667,0.556667,"[0.705, 0.7116666666666667, 0.7483333333333333...",0.761667
4,0.608333,"[0.805, 0.8116666666666666, 0.6583333333333333...",0.830000,0.483333,"[0.535, 0.66, 0.75, 0.835, 0.6083333333333333,...",0.835000,0.465000,"[0.8, 0.73, 0.49166666666666664, 0.605, 0.4666...",0.800000,0.681667,...,0.493333,0.461667,"[0.79, 0.6433333333333333, 0.605, 0.4816666666...",0.793333,0.473333,"[0.47333333333333333, 0.47333333333333333, 0.4...",0.473333,0.293333,"[0.325, 0.62, 0.36666666666666664, 0.35, 0.316...",0.620000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.483333,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN
224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.480000,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN
225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.475000,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.486667,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN


In [211]:
len("_scores_0")

9

In [212]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    print(aggregation_method)
    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-21]]
    sub_filtered_rel_cols=[c for c in rel_col if "pre_training" in c]
    result_dict=dict()
    result_dict["by_node_results_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-13]]
    sub_filtered_rel_cols=[c for c in rel_col if "best_score" in c]
    result_dict["by_node_results_best_score"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_best_score"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-9]]
    sub_filtered_rel_cols=[c for c in rel_col if "scores" in c]
    by_node_result_average_dict=dict()
    by_node_result_std_dict=dict()
    by_node_retraining_count_dict=dict()
    by_node_training_count_dict=dict()
    for col in sub_filtered_rel_cols:
        by_node_result_average_dict[col.split("_")[-1]]=np.mean(np.concatenate(training_scores_df[col].dropna().values))
        by_node_result_std_dict[col.split("_")[-1]]=np.std(np.concatenate(training_scores_df[col].dropna().values))
        by_node_training_count_dict[col.split("_")[-1]]=len(np.concatenate(training_scores_df[col].dropna().values))
        by_node_retraining_count_dict[col.split("_")[-1]]= len(training_scores_df[col].dropna().values)
    result_dict["by_node_results_average_scores"]=by_node_result_average_dict
    result_dict["total_average_average_scores"]=np.mean([val for val in by_node_result_average_dict.values()])
    result_dict["by_node_results_std_scores"]=by_node_result_std_dict
    result_dict["total_average_std_scores"]=np.mean([val for val in by_node_result_std_dict.values()])
    result_dict["by_node_results_retraining_count"]=by_node_retraining_count_dict
    result_dict["total_retraining_count"]=np.sum([val for val in by_node_retraining_count_dict.values()])
    result_dict["by_node_results_training_count"]=by_node_training_count_dict
    result_dict["total_training_count"]=np.sum([val for val in by_node_training_count_dict.values()])
    average_score_dict[aggregation_method]=result_dict

MEDIAN_AGGREGATION
DYNAMIC_CLIPPED_AVERAGE_AGGREGATION
WEIGHTED_AVERAGE_RECENCY
AVERAGE_AGGREGATION
WEIGHTED_AVERAGE_PERFORMANCE_SCORES
NO_FEDERATED_LEARNING
FIXED_CLIPPED_AVERAGE_AGGREGATION
WEIGHTED_AVERAGE_PUNISHING_UPDATES


In [213]:
total_average_pre_training = []
total_average_best_score = []
total_average_std_scores = []
total_retraining_count = []
total_training_count = []
total_average_average_scores = []
aggregation_names=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    total_average_pre_training.append(val["total_average_pre_training"])
    total_average_best_score.append(val["total_average_best_score"])
    total_average_std_scores.append(val["total_average_std_scores"])
    total_retraining_count.append(val["total_retraining_count"])
    total_training_count.append(val["total_training_count"])
    total_average_average_scores.append(val["total_average_average_scores"])
print(list(zip(aggregation_names,total_training_count)))
sorted_tuples = sorted(
    zip(
        aggregation_names,
        total_average_pre_training,
        total_average_best_score,
        total_average_std_scores,
        total_retraining_count,
        total_training_count,
        total_average_average_scores,
    ),
    key=lambda x: x[2],
)
aggregation_names = [val[0] for val in sorted_tuples]
total_average_pre_training = [val[1] for val in sorted_tuples]
total_average_best_score = [val[2] for val in sorted_tuples]
total_average_std_scores = [val[3] for val in sorted_tuples]
total_retraining_count = [val[4] for val in sorted_tuples]
total_training_count = [val[5] for val in sorted_tuples]
total_average_average_scores = [val[6] for val in sorted_tuples]

[('MEDIAN_AGGREGATION', 999), ('DYNAMIC_CLIPPED_AVERAGE_AGGREGATION', 1175), ('WEIGHTED_AVERAGE_RECENCY', 1224), ('AVERAGE_AGGREGATION', 1036), ('WEIGHTED_AVERAGE_PERFORMANCE_SCORES', 2221), ('NO_FEDERATED_LEARNING', 1691), ('FIXED_CLIPPED_AVERAGE_AGGREGATION', 1489), ('WEIGHTED_AVERAGE_PUNISHING_UPDATES', 3338)]


In [214]:
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Punish updates',
 'Fixed clip',
 'Median',
 'Simple average',
 'Dynamic clip',
 'Performance weighted',
 'Receny weighted',
 'No FL']

In [215]:
fig=go.Figure()
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_pre_training,name="Pre-training"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_best_score,name="Best"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_average_scores,name="Average"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_average_std_scores,name="Training std"))
fig.update_layout(template="none", title=f"Training results by {plot_file_term} for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Training_results_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()

In [216]:
fig=go.Figure()
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_training_count,name="Trainings"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=total_retraining_count,name="Re-trainings"))
fig.update_layout(template="none", title=f"Training cost by {plot_file_term} methods for {NODE_COUNT} nodes",
                  width=800,height=400,legend=dict( orientation="h",y=1.2))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Number of executions")
fig.write_image(f"./../figs/Training_cost_by_{plot_file_term}_{NODE_COUNT}_nodes.svg")
fig.show()